In [1]:
import pandas as pd
import numpy as np
import os

import plotly.graph_objs as go

In [2]:
train_df = pd.read_csv('../train.csv')

In [3]:
train_df.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0


### Looking at the number of duplicates (using plotly 3.0!)

In [4]:
dup_df = train_df.\
         groupby('is_duplicate',
                 as_index = False)\
         ['id'].\
         count()


In [5]:
fig = go.FigureWidget(
    data = [go.Bar(
        x=dup_df['is_duplicate'],
        y=dup_df['id']
    )],
    layout=go.Layout(
        title="Number of records for each 'is_duplicate' class",
        titlefont=dict(
            color='rgb(230,230,230)'
        ),
        xaxis=dict(
            title='is duplicate',
            color='rgb(230,230,230)'
        ),
        yaxis=dict(
            title='Count',
            color='rgb(230,230,230)'
        ),
        paper_bgcolor = 'rgb(44,48,60)',
        plot_bgcolor = 'rgb(44,48,60)'
    )
)


In [6]:
fig

FigureWidget({
    'data': [{'type': 'bar',
              'uid': '3f895810-829a-11e8-acfb-bc8385288ae2',
              'x': array([0., 1.]),
              'y': array([255027., 149263.])}],
    'layout': {'paper_bgcolor': 'rgb(44,48,60)',
               'plot_bgcolor': 'rgb(44,48,60)',
               'title': "Number of records for each 'is_duplicate' class",
               'titlefont': {'color': 'rgb(230,230,230)'},
               'xaxis': {'color': 'rgb(230,230,230)', 'title': 'is duplicate'},
               'yaxis': {'color': 'rgb(230,230,230)', 'title': 'Count'}}
})

### Let's take a look at how many times each question appears

In [7]:
train_df.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'], dtype='object')

In [14]:
questions = pd.concat([
                train_df[['qid1','is_duplicate']],
                train_df[['qid2','is_duplicate']].\
                    rename(columns={'qid2' : 'qid1'})
                ],
                axis=0
)

In [18]:
questions = pd.DataFrame(questions).\
    rename(columns={'qid1':'qid'})
# adding dummy indicator for aggregation
questions['ind'] = 1

In [77]:
questions_w_duplicates = questions.\
                        groupby(['qid','is_duplicate'],
                                 as_index = False)\
                        ['ind'].\
                        count().\
                        sort_values('ind',
                                     ascending=False
                        ).\
                        rename(columns={'ind':'count'}).\
                        merge(
                            pd.DataFrame([{'qid' : x, 'is_duplicate' : 0} 
                                          for x in questions['qid'].unique().tolist()] +\
                                         [{'qid' : x, 'is_duplicate' : 1} 
                                          for x in questions['qid'].unique().tolist()]
                            ),
                            how='right',
                            on=['qid','is_duplicate']
                        )

questions_w_duplicates.fillna(0)

questions_w_duplicates.head(5)

,qid,is_duplicate,count
0,30782,0,120.0
1,2559,0,118.0
2,4044,0,85.0
3,17978,0,77.0
4,14376,0,69.0


In [79]:
question_counts = questions.\
                    groupby('qid',
                             as_index = False)\
                    ['ind'].\
                    count().\
                    rename(columns={'ind':'total'}).\
                    merge(
                        questions_w_duplicates,
                        on='qid'
                    ).\
                    sort_values('total',
                                 ascending=False
                    )
                    
question_counts.head(3)

,qid,total,is_duplicate,count
5116,2559,157,0,118.0
5117,2559,157,1,39.0
61562,30782,120,0,120.0


In [83]:
# set this parameter to be whatever you want.
num_questions = 50

q_fig = go.FigureWidget(
    data = [
    go.Bar(
        x=question_counts\
          [question_counts['is_duplicate'] == 0]\
          ['qid'].\
          apply(lambda x: 'qid: %i' % x).\
          head(num_questions),
        y=question_counts\
          [question_counts['is_duplicate'] == 0]\
          ['count'].\
          head(num_questions),
        name='non duplicate'
    ),
    go.Bar(
        x=question_counts\
          [question_counts['is_duplicate'] == 1]\
          ['qid'].\
          apply(lambda x: 'qid: %i' % x).\
          head(num_questions),
        y=question_counts\
          [question_counts['is_duplicate'] == 1]\
          ['count'].\
          head(num_questions),
        name='is not duplicate'
    )
    ],
    layout=go.Layout(
        title="Number of questions appearing in the training set for each qid (top %i questions)" % num_questions,
        titlefont=dict(
            color='rgb(230,230,230)'
        ),
        xaxis=dict(
            title='qid',
            color='rgb(230,230,230)'
        ),
        yaxis=dict(
            title='Count',
            color='rgb(230,230,230)'
        ),
        barmode='stack',
        legend=dict(
            font=dict(
                color='rgb(230,230,230)'
            )
        ),
        paper_bgcolor = 'rgb(44,48,60)',
        plot_bgcolor = 'rgb(44,48,60)'
    )
)


In [84]:
q_fig

FigureWidget({
    'data': [{'name': 'non duplicate',
              'type': 'bar',
              'uid': 'ac93c800-82a2-11e8-bbe8-bc8385288ae2',
              'x': array(['qid: 2559', 'qid: 30782', 'qid: 4044', 'qid: 2561', 'qid: 14376',
                          'qid: 17978', 'qid: 2675', 'qid: 28764', 'qid: 1772', 'qid: 24555',
                          'qid: 18753', 'qid: 4018', 'qid: 28133', 'qid: 3595', 'qid: 18531',
                          'qid: 4951', 'qid: 19621', 'qid: 10024', 'qid: 38', 'qid: 13748',
                          'qid: 10330', 'qid: 18296', 'qid: 11264', 'qid: 20190', 'qid: 6551',
                          'qid: 6749', 'qid: 8461', 'qid: 33412', 'qid: 691', 'qid: 12639',
                          'qid: 2322', 'qid: 3938', 'qid: 1885', 'qid: 10382', 'qid: 4367',
                          'qid: 58173', 'qid: 11588', 'qid: 1749', 'qid: 7801', 'qid: 17761',
                          'qid: 21693', 'qid: 2439', 'qid: 11397', 'qid: 27618', 'qid: 65693',
                          'qid: 25984', 'qid: 2921', 'qid: 14110', 'qid: 6100', 'qid: 10433'],
                         dtype=object),
              'y': array([118., 120.,  85.,  49.,  69.,  77.,  21.,  44.,  44.,  50.,  36.,   4.,
                           nan,  nan,  51.,   1.,  11.,  40.,  nan,  nan,  nan,  nan,   1.,  nan,
                           nan,  nan,  nan,  nan,  31.,  nan,  nan,   1.,   2.,  32.,  nan,   1.,
                            1.,  nan,   1.,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
                            1.,   4.])},
             {'name': 'is not duplicate',
              'type': 'bar',
              'uid': 'ac93c801-82a2-11e8-b093-bc8385288ae2',
              'x': array(['qid: 2559', 'qid: 30782', 'qid: 4044', 'qid: 2561', 'qid: 14376',
                          'qid: 17978', 'qid: 2675', 'qid: 28764', 'qid: 1772', 'qid: 24555',
                          'qid: 18753', 'qid: 4018', 'qid: 28133', 'qid: 18531', 'qid: 4951',
                          'qid: 3595', 'qid: 19621', 'qid: 38', 'qid: 10330', 'qid: 10024',
                          'qid: 13748', 'qid: 18296', 'qid: 11264', 'qid: 6551', 'qid: 20190',
                          'qid: 3938', 'qid: 6749', 'qid: 8461', 'qid: 33412', 'qid: 691',
                          'qid: 2322', 'qid: 12639', 'qid: 58173', 'qid: 10382', 'qid: 4367',
                          'qid: 11588', 'qid: 1885', 'qid: 1749', 'qid: 11397', 'qid: 21693',
                          'qid: 17761', 'qid: 7801', 'qid: 65693', 'qid: 25984', 'qid: 27618',
                          'qid: 2439', 'qid: 6100', 'qid: 2921', 'qid: 14110', 'qid: 10433'],
                         dtype=object),
              'y': array([39., nan, 26., 39., 10., nan, 47., 22., 19., 11., 22., 53., 56.,  4.,
                          54., 55., 44., 54., 54., 14., 54., 53., 51., 52., 52., 50., 51., 51.,
                          51., 20., 51., 51., 49., 18., 50., 49., 48., 49., 49., 49., 49., 48.,
                          49., 49., 49., 49., 47., 48., 48., 43.])}],
    'layout': {'barmode': 'stack',
               'legend': {'font': {'color': 'rgb(230,230,230)'}},
               'paper_bgcolor': 'rgb(44,48,60)',
               'plot_bgcolor': 'rgb(44,48,60)',
               'title': ('Number of questions appearing ' ... 'or each qid (top 50 questions)'),
               'titlefont': {'color': 'rgb(230,230,230)'},
               'xaxis': {'color': 'rgb(230,230,230)', 'title': 'qid'},
               'yaxis': {'color': 'rgb(230,230,230)', 'title': 'Count'}}
})

In [85]:
q_fig.layout.bargap = 0.3

# So some questions appear multiple times in the dataset, yet the 'is_duplicate' field is not distributed evenly among questions

Lets take a look at qid: 2559


In [87]:
train_df[(train_df['qid1'] == 2559)].head(10)

,id,qid1,qid2,question1,question2,is_duplicate
16138,16138,30781,30782,If I link my Instagram account to my Facebook ...,How can you look at someone's private Instagra...,0
16183,16183,30866,30782,Can you view pictures on Instagram without an ...,How can you look at someone's private Instagra...,0
17232,17232,32775,30782,How many Oriya girls wear mini skirts?,How can you look at someone's private Instagra...,0
20239,20239,38203,30782,Why did Symbian fail?,How can you look at someone's private Instagra...,0
22376,22376,41999,30782,How do I see followers on someone's private In...,How can you look at someone's private Instagra...,0
29583,29583,54720,30782,Why did my crush followed my cousin's private ...,How can you look at someone's private Instagra...,0
39470,39470,71541,30782,Is there a way to view a private Instagram?,How can you look at someone's private Instagra...,0
43079,43079,77543,30782,How do I recover a deleted Instagram name?,How can you look at someone's private Instagra...,0
47124,47124,58696,30782,Is it possible to view someone's private Insta...,How can you look at someone's private Instagra...,0
47824,47824,48228,30782,How do I open a private Instagram account?,How can you look at someone's private Instagra...,0


These are seemingly weight-loss questions.

How about 30782 (no duplicates) or 38 (all duplicates)

In [89]:
train_df[(train_df['qid2'] == 30782)].head(5)

,id,qid1,qid2,question1,question2,is_duplicate
16138,16138,30781,30782,If I link my Instagram account to my Facebook ...,How can you look at someone's private Instagra...,0
16183,16183,30866,30782,Can you view pictures on Instagram without an ...,How can you look at someone's private Instagra...,0
17232,17232,32775,30782,How many Oriya girls wear mini skirts?,How can you look at someone's private Instagra...,0
20239,20239,38203,30782,Why did Symbian fail?,How can you look at someone's private Instagra...,0
22376,22376,41999,30782,How do I see followers on someone's private In...,How can you look at someone's private Instagra...,0


In [88]:
train_df[(train_df['qid2'] == 38)].head(5)

,id,qid1,qid2,question1,question2,is_duplicate
18,18,37,38,Why are so many Quora users posting questions ...,Why do people ask Quora questions which can be...,1
19484,19484,4950,38,"How come people on Quora ask questions here, w...",Why do people ask Quora questions which can be...,1
22076,22076,17507,38,Why do so many people post questions on Quora ...,Why do people ask Quora questions which can be...,1
25791,25791,48053,38,Why do people keep asking questions on Quora e...,Why do people ask Quora questions which can be...,1
26372,26372,21713,38,Why do so many people ask questions on Quora i...,Why do people ask Quora questions which can be...,1


So a very specific question about Instagram privacy vs a generic question about Quora questions.

Let's build a 'corpus' out of the quora questions

In [15]:
corpus = pd.concat([
            train_df[['qid1','question1']],
            train_df[['qid2','question2']].\
                rename(columns={'qid2':'qid1',
                                'question2':'question1'}
                )],
            axis=0
        )

corpus = corpus.\
         rename(columns={'qid1':'qid',
                         'question1':'question'}
         ).\
         drop_duplicates()

In [16]:
corpus.head(3)

,qid,question
0,1,What is the step by step guide to invest in sh...
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,5,How can I increase the speed of my internet co...


## cleaning with nltk

In [17]:
import nltk

In [45]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string

In [19]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

### There may be spelling mistakes in the questions, but will try to find these later

In [20]:
def clean(doc):
    doc = str(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [21]:
corpus['question'] = corpus['question'].apply(clean)

## Looking at some of the words that appear most frequently

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
vectorizer = CountVectorizer(min_df=1)

vectorizer = CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=np.float32, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [93]:
vector_matrix = vectorizer.fit_transform(corpus['question'].tolist()[:10000])

In [94]:
appearences = vector_matrix.sum(axis=0)
feat_names = vectorizer.get_feature_names()

In [107]:
word_counts = [{'word' : feat_names[i],
                'count' : appearences[0,i]} for i in range(np.shape(appearences)[1])]

word_counts = pd.DataFrame(word_counts).\
              sort_values('count', ascending=False)

In [138]:
# set this parameter to be whatever you want.
num_words = 20

w_fig = go.FigureWidget(
    data = [go.Bar(
        y=word_counts.\
          head(num_questions).\
          sort_values('count')\
          ['word'],
        x=word_counts.\
          head(num_questions).\
          sort_values('count')\
          ['count'],
        orientation = 'h'
    )],
    layout=go.Layout(
        title="Number of times each word appears in the corpus for %i words (descending order)" % num_questions,
        titlefont=dict(
            color='rgb(230,230,230)'
        ),
        xaxis=dict(
            title='qid',
            color='rgb(230,230,230)'
        ),
        yaxis=dict(
            title='Count',
            color='rgb(230,230,230)'
        ),
        paper_bgcolor = 'rgb(44,48,60)',
        plot_bgcolor = 'rgb(44,48,60)',
        font=dict(size=8),
        height=700
    )
)

In [139]:
w_fig

FigureWidget({
    'data': [{'orientation': 'h',
              'type': 'bar',
              'uid': 'bb2f106c-8194-11e8-9434-bc8385288ae2',
              'x': array([100., 100., 102., 103., 103., 109., 111., 115., 118., 119., 123., 123.,
                          125., 126., 127., 127., 128., 129., 130., 133., 133., 139., 140., 144.,
                          145., 146., 146., 146., 148., 151., 156., 156., 158., 158., 166., 176.,
                          181., 191., 193., 220., 251., 265., 265., 266., 288., 290., 316., 332.,
                          481., 851.]),
              'y': array(['go', 'account', 'it', 'company', 'better', 'note', 'become', 'want',
                          'world', 'first', 'start', 'ever', 'girl', 'book', 'movie', 'mean',
                          'without', 'find', 'many', 'job', 'trump', 'money', 'day', 'work',
                          'someone', 'thing', 'question', 'new', 'much', 'use', 'whats', 'indian',
                          'year', 'think', 'know', 'time', 'learn', 'life', 'quora', 'difference',
                          'make', 'would', 'way', 'one', 'people', 'good', 'like', 'india', 'get',
                          'best'], dtype=object)}],
    'layout': {'font': {'size': 8},
               'height': 700,
               'paper_bgcolor': 'rgb(44,48,60)',
               'plot_bgcolor': 'rgb(44,48,60)',
               'title': 'Number of times each word appears in the corpus for 50 words (descending order)',
               'titlefont': {'color': 'rgb(230,230,230)'},
               'xaxis': {'color': 'rgb(230,230,230)', 'title': 'qid'},
               'yaxis': {'color': 'rgb(230,230,230)', 'title': 'Count'}}
})

## Some observations from the data above:
* some words should be treated in the same foot (e.g. india and indian)
* best is the most dominant word in the corpus, questions are probably something like 'what is the best...'
* questions that are hypothetical may be characterised as having 'would'

...

## Using gensim to run a LDA model

In [22]:
from gensim import corpora, models

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



Taking the first 10,000 records just to see what happens.

In [23]:
dictionary = corpora.Dictionary([x.split() for x in corpus['question'].tolist()[:10000]])

In [24]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc 
                   in [x.split() for x in corpus['question'].tolist()[:10000]]
                  ]

In [41]:
Lda = models.ldamodel.LdaModel
num_topics = 30
# Running and Trainign LDA model on the document term matrix.
# Use defaul priors, no callbacks yet.
ldamodel = Lda(corpus = doc_term_matrix, 
               num_topics=num_topics, 
               id2word = dictionary,
               distributed=False,
               passes=50,
               minimum_probability = 0.05
)

### Printing some topics.

In [42]:
print(ldamodel.print_topics(num_topics=num_topics, num_words=5))

[(0, '0.052*"thing" + 0.042*"know" + 0.035*"year" + 0.033*"take" + 0.033*"new"'), (1, '0.119*"make" + 0.036*"money" + 0.020*"american" + 0.017*"happens" + 0.016*"option"'), (2, '0.160*"best" + 0.091*"way" + 0.033*"online" + 0.024*"company" + 0.019*"much"'), (3, '0.046*"want" + 0.040*"engineering" + 0.037*"woman" + 0.028*"me" + 0.025*"do"'), (4, '0.053*"trump" + 0.035*"2016" + 0.030*"donald" + 0.021*"hillary" + 0.021*"clinton"'), (5, '0.026*"get" + 0.022*"you" + 0.022*"not" + 0.019*"skill" + 0.016*"energy"'), (6, '0.133*"india" + 0.030*"war" + 0.027*"travel" + 0.025*"increase" + 0.016*"world"'), (7, '0.040*"without" + 0.034*"buy" + 0.033*"phone" + 0.025*"data" + 0.018*"android"'), (8, '0.118*"people" + 0.078*"life" + 0.040*"love" + 0.031*"stop" + 0.017*"window"'), (9, '0.052*"find" + 0.033*"system" + 0.032*"old" + 0.028*"facebook" + 0.028*"business"'), (10, '0.074*"difference" + 0.053*"many" + 0.038*"english" + 0.033*"school" + 0.024*"type"'), (11, '0.140*"good" + 0.036*"example" + 0.03